In [1]:
from tensorflow import keras
from keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_datasets as tfds

import os, shutil, pathlib, glob
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import pandas as pd
import random
SEED = 4747
random.seed(SEED)

We are uploading the kaggle json file to faciliate the process of obtaining the dataset of histopathological images

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bluekingnfx","key":"a50380852bb480d73c7f7e43389358c7"}'}

Initializing Kaggle directory in google collab and moving the json into the kaggle folder to install dataset

In [3]:
#Create "~/.kaggle" directory
!mkdir ~/.kaggle

#Move "kaggle.json" file to this directory
!mv kaggle.json ~/.kaggle/

Giving the write access to the folder created in linux os of google collab hosted in cloud

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

Downloading the dataset with API command

In [5]:
!kaggle datasets download -d paultimothymooney/breast-histopathology-images

100% 3.10G/3.10G [01:31<00:00, 41.3MB/s]
100% 3.10G/3.10G [01:31<00:00, 36.3MB/s]


Create a directory "dataset" to unzip the file there

In [6]:
!mkdir dataset
!ls

breast-histopathology-images.zip  dataset  sample_data


Unziping the dataset

In [ ]:
!unzip -qq "/content/breast-histopathology-images.zip" -d "/content/dataset"

Total directories (that is, total number of patients)

In [ ]:
dirs = os.listdir("/content/dataset/")
print("Total number of directories are =",len(dirs))

The dataset assigns a unique filename structure to each image, like:
```
u_xX_yY_classC.png

```
For example:
```
10253_idx5_x1351_y1101_class0.png
```

- "u" is patient id
- "u" is the patient ID (10253_idx5),
- "X" is the x-coordinate of where this patch was cropped from,
- "Y" is the y-coordinate of where this patch was cropped from, and
- "C" indicates the class where 0 is non-IDC and 1 is IDC.

In [ ]:

def plot_random_patches(clas):
    fig, axes = plt.subplots(7,7,figsize=(10,10))
    for ax in axes.flatten():
        #Get a random patient id
        patient_id = random.choice(dirs)

        path = f"/content/dataset/{patient_id}/{clas}/*.png"
        #Read all the files' path present in "path" directory
        paths = glob.glob(path)
        #Select a random file path
        p = random.choice(paths)
        #Get the path with semantics appropriate for underlying OS
        p = pathlib.Path(p)
        #Load the image
        img = load_img(p)
        #Plot theimage
        ax.imshow(img)
        #Set image properties
        ax.set_title(label=patient_id, fontdict={"fontsize":8})
        ax.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)

    #Use different image title for cancer and normal tissues
    if clas == 1:
        fig.suptitle('IDC+ patches\n(Cancer tissues)', fontsize=16)
    else:
        fig.suptitle('IDC- patches\n(Noraml tissues)', fontsize=16)

Plot random images of IDC+ / cancer tissues

In [ ]:

plot_random_patches(clas=1)

In [ ]:
"""
The core functionality: For each paitent (directory)
thier corresponding 0 and 1 samples are seggregated and randomized,
then the classes for each iteration is extended to main arrays of class_0, class_1
This function is built to improve random sampling.
"""

def get_image_path_list():
    #Create empty lists to store the paths of class 0 and class 1 images
    class_0 = []
    class_1 = []
    for dir in dirs:
        #Read class 0 and class 1 images' paths for a given patient
        # and store them it their respective list
        c_0 = glob.glob(f"/content/dataset/{dir}/0/*.png")
        c_1 = glob.glob(f"/content/dataset/{dir}/1/*.png")

        random.shuffle(c_0)
        random.shuffle(c_1)

        #Add the class 0 and class 1 images' paths for a given patient
        # to the main list
        class_0.extend(c_0)
        class_1.extend(c_1)

        random.shuffle(class_0)
        random.shuffle(class_1)

    #Shuffle the paths lists randomly
    random.shuffle(class_0), random.shuffle(class_0)
    random.shuffle(class_1), random.shuffle(class_1)

    #Return the class_0 and class_1 lists
    return class_0, class_1

In [ ]:
'''
Core functionality is to create a train and test directory under the new_dataset main directory,
the seggregation of the image files of specific class is defined by specific threshold value,
default in create_dataset function, about 80% of data is used for training.

'''

def create_dataset(train_split=0.80):

    train0_dir = "/content/new_dataset/train/0"
    train1_dir = "/content/new_dataset/train/1"
    os.makedirs(train0_dir)
    os.mkdir(train1_dir)

    test0_dir = "/content/new_dataset/test/0"
    test1_dir = "/content/new_dataset/test/1"
    os.makedirs(test0_dir)
    os.mkdir(test1_dir)

    class0, class1 = get_image_path_list()

    random.shuffle(class0)
    random.shuffle(class1)

    total_img0 = len(class0)
    total_img1 = len(class1)

    train0_thresh = int(total_img0 * train_split)
    train1_thresh = int(total_img1 * train_split)

    for i in range(total_img0):
        path = class0[i]
        if i < train0_thresh:
            shutil.copy(src=path, dst= train0_dir)
        else:
            shutil.copy(src=path, dst= test0_dir)

    for i in range(total_img1):
        path = class1[i]
        if i < train1_thresh:
            shutil.copy(src=path, dst= train1_dir)
        else:
            shutil.copy(src=path, dst= test1_dir)

In [ ]:
create_dataset()

Checkpoint: We new_dataset directory of structure train and test sub directory, this indeed contains 0 and 1 directory of 0 and 1 images. Train: 80% data, test: 20%.

In [ ]:
class0, class1 = get_image_path_list()

print("80% images of class 0 =", int(len(class0)*0.80))
print("80% images in class 1 =", int(len(class1)*0.80))

train_data_size = glob.glob("/content/new_dataset/train/**/*.png")
print("Training dataset size =",len(train_data_size))

Image_dataset_from directory helps to create a image dataset from class labels infered from the subdirectories, image_size is resized to 50x50 and 32 images are processed once.

Subnet argument in validation_dataset is used to return all the images that were ommited in training dataset to validation dataset

In [ ]:
image_size = (50,50)
batch_size = 32
train_dataset = image_dataset_from_directory(
    "/content/new_dataset/train",
    labels = "inferred",
    label_mode = 'int',
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=image_size,
    batch_size=batch_size,
)


validation_dataset =image_dataset_from_directory(
    "/content/new_dataset/train",
    labels = "inferred",
    label_mode = 'int',
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=image_size,
    batch_size=batch_size,
)

Same process is carried for testing dataset without subnetting

In [ ]:
test_dataset = image_dataset_from_directory(
    "/content/new_dataset/test",
    labels = "inferred",
    label_mode = 'int',
    seed=SEED,
    image_size=image_size,
    batch_size=batch_size,
)

Since neural networks should consume lot of images to cover up/recoginize, since over dataset of training is just 2 lakhs+ it is minimal, this tend to make the model more or less overfit, to overcome this, we going to use data augmentation and dropout layer techniques.

Data agumentation - Is the process of creating additional images by resizing and magnizing the already existing images, there by increasing the size of the exisiting dataset

Dropout layer - Disabling certain nodal layers to combat the team effect, putting more burdern certain layers to have a significant effect.

For modeling, the keras sequential architure is utilized, Pulling in layers required to form a neural network of own.

Combat overfitting

In [ ]:
inputs = keras.Input(shape=(50, 50, 3))
x = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.3),
        layers.RandomZoom(0.3)
    ]
)(inputs)
x = layers.Rescaling(1./255)(x)

Conventions
- Going to use the generally used architecture,
Convultional neural layer -> Batch normalization -> Activation -> pooling

Neural set to identify of 256 filters/features initally, larger number because smaller dimension of the image.
Batch normalization, normalizing the input and activating the nodes

Pooling -> Amplifing the patterns/features

Three layers same process flow to improve better recoginization. Only 3 cause of computational cost







In [ ]:
x = layers.Conv2D(filters=256, kernel_size=3, use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.Conv2D(filters=256, kernel_size=3, use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation("relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)

Flatting the multi-dimension vector to single dimensional, passed to dense layer (fully connected NN) and to the output layer assigns 0 or 1

In [ ]:
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)


Connecting to input and output layer

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

input_2: The input image layer taking images of shape (50, 50, 3)

sequential_1: Empty sequential model, does nothing
rescaling_1: Rescales input pixels from 0-255 to 0-1.

conv2d_4: 2D Convolution layer with 256 filters of kernel size 3

batch_normalization_4: Normalizes activations from conv layer

activation_4: ReLU activation function

conv2d_5, batch_normalization_5 etc: Repeats convolution, batch norm, activation blocks.

max_pooling2d_3: Max pooling to downsample spatially by half.

flatten_1: Flattens final conv feature maps to 1D vector

dropout_1: Applies dropout regularization

dense_4, dense_5 etc: Fully connected layers outputting class predictions.

Total number of edges in the neural network are about 39lakhish and on - trainable params are of 2048 contributed by normalization layers.

Compiling before training,

Telling model what things must be used,

loss function is to binary_crossentropy -> assigning 0 or 1
optimizer - Update weights during training

metrices - To been considered while training

In [ ]:
model.compile(loss="binary_crossentropy",
               optimizer="rmsprop",
               metrics=["accuracy"])

Stopping condition when to save the model


In [ ]:
callbacks = [
            keras.callbacks.ModelCheckpoint(filepath="model.keras",
                                            save_best_only=True,
                                            monitor="val_loss")
            ]

Training the model

In [ ]:
fit_model = model.fit(
                    train_dataset,
                    epochs=4,
                    validation_data=validation_dataset,
                    callbacks=callbacks
                   )

In [ ]:
test_model = keras.models.load_model("/content/CanDetect.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")